In [11]:
import pandas as pd
import numpy as np
import keras

# Read the CSV and Perform Basic Data Cleaning

In [86]:
df = pd.read_csv("data_5hoods_datetime.csv")

In [87]:
#Select columns of interst
df = df[["Beat",
    "Occur Time",
    "Neighborhood",
    "Location Type",
    "Latitude",
    "Longitude",
    "Occur Year",
    "Occur Month",
    "Occur Day",
    "Occur DayofWeek",
    "UCR Literal"]]
#df.dropna(subset=['Location Type', 'Occur Time'])
df.dropna(subset=['Beat'], inplace=True)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
y = df["UCR Literal"]
X = df.drop(columns=["UCR Literal"])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 
X['Neighborhood']= le.fit_transform(X['Neighborhood']) 

In [15]:
X.shape

(61961, 10)

# Create a Train Test Split

Use `UCR Literal` for the y values

In [16]:
#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
X_train.shape

(41513, 10)

In [153]:
X_train

,Beat,Occur Time,Neighborhood,Location Type,Latitude,Longitude,Occur Year,Occur Month,Occur Day,Occur DayofWeek
36637,604.0,1630,4,21.0,33.76142,-84.36492,2013.0,12.0,16.0,0.0
52369,505.0,1230,3,13.0,33.77448,-84.38263,2017.0,9.0,3.0,6.0
9562,505.0,200,3,13.0,33.77454,-84.37885,2009.0,1.0,4.0,6.0
21715,511.0,930,2,99.0,33.75281,-84.39197,2011.0,1.0,25.0,1.0
40637,604.0,2030,4,18.0,33.75772,-84.37067,2014.0,10.0,11.0,5.0
...,...,...,...,...,...,...,...,...,...,...
54343,503.0,1100,3,18.0,33.78045,-84.38923,2018.0,2.0,25.0,6.0
38158,509.0,2250,2,18.0,33.76020,-84.38554,2014.0,4.0,25.0,4.0
860,208.0,1730,1,8.0,33.85166,-84.36235,2009.0,12.0,10.0,3.0
15795,512.0,2100,2,18.0,33.75135,-84.39257,2009.0,12.0,16.0,2.0


In [162]:
X_train.to_numpy()

array([[6.04e+02, 1.63e+03, 4.00e+00, ..., 1.20e+01, 1.60e+01, 0.00e+00],
       [5.05e+02, 1.23e+03, 3.00e+00, ..., 9.00e+00, 3.00e+00, 6.00e+00],
       [5.05e+02, 2.00e+02, 3.00e+00, ..., 1.00e+00, 4.00e+00, 6.00e+00],
       ...,
       [2.08e+02, 1.73e+03, 1.00e+00, ..., 1.20e+01, 1.00e+01, 3.00e+00],
       [5.12e+02, 2.10e+03, 2.00e+00, ..., 1.20e+01, 1.60e+01, 2.00e+00],
       [5.10e+02, 1.83e+03, 2.00e+00, ..., 9.00e+00, 7.00e+00, 4.00e+00]])

In [163]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train.to_numpy())#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)
np.save('../models/DL/dl_xscaler.npy', X_scaler.scale_)

# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [24]:
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder

In [25]:
y_train

36637    LARCENY-FROM VEHICLE
52369    LARCENY-FROM VEHICLE
9562       ROBBERY-PEDESTRIAN
21715         BURGLARY-NONRES
40637      ROBBERY-PEDESTRIAN
                 ...         
54343              AUTO THEFT
38158    LARCENY-FROM VEHICLE
860       LARCENY-NON VEHICLE
15795      ROBBERY-PEDESTRIAN
56422     LARCENY-NON VEHICLE
Name: UCR Literal, Length: 41513, dtype: object

In [26]:
label_encoder = LabelEncoder()# Should probably use OneHotEncoder instead
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


In [128]:
import joblib
joblib.dump(label_encoder, '../models/DL/scaler.sav')

['../models/DL/scaler.sav']

In [27]:
label_encoder.classes_

array(['AGG ASSAULT', 'AUTO THEFT', 'BURGLARY-NONRES',
       'BURGLARY-RESIDENCE', 'HOMICIDE', 'LARCENY-FROM VEHICLE',
       'LARCENY-NON VEHICLE', 'ROBBERY-COMMERCIAL', 'ROBBERY-PEDESTRIAN',
       'ROBBERY-RESIDENCE'], dtype=object)

In [28]:
#https://stackoverflow.com/questions/28656736/using-scikits-labelencoder-correctly-across-multiple-programs
#np.save('dl_classes.npy', label_encoder.classes_)

# Train the Model

In [114]:
model = Sequential()
y_num_unique = y_train_categorical.shape[1]
X_num_col = X_test_scaled.shape[1]
model.add(Dense(units=200, activation='relu', input_dim=X_num_col))
model.add(Dense(units=200, activation='relu', input_dim=X_num_col))
model.add(Dense(units=200, activation='relu', input_dim=X_num_col))
model.add(Dense(units=y_num_unique, activation='softmax'))

In [115]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [116]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 200)               2200      
_________________________________________________________________
dense_8 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_9 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                2010      
Total params: 84,610
Trainable params: 84,610
Non-trainable params: 0
_________________________________________________________________


In [117]:
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
#https://keras.rstudio.com/reference/fit.html
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks, #List of `keras.callbacks.Callback` instances.
    epochs=60, #Integer. Number of epochs to train the model.
    shuffle=True,#Boolean (whether to shuffle the training data before each epoch)
    verbose=1, #0 = silent, 1 = progress bar, 2 = one line per epoch.
    #view_metrics = 'auto'
)

Epoch 1/60
1298/1298 [==============================] - 1s 828us/step - loss: 1.5120 - accuracy: 0.4581
Epoch 2/60
1298/1298 [==============================] - 1s 889us/step - loss: 1.2195 - accuracy: 0.6039
Epoch 3/60
1298/1298 [==============================] - 1s 878us/step - loss: 1.1533 - accuracy: 0.6199
Epoch 4/60
1298/1298 [==============================] - 1s 845us/step - loss: 1.1307 - accuracy: 0.6264
Epoch 5/60
1298/1298 [==============================] - 1s 831us/step - loss: 1.1110 - accuracy: 0.6348
Epoch 6/60
1298/1298 [==============================] - 1s 875us/step - loss: 1.1038 - accuracy: 0.6345
Epoch 7/60
1298/1298 [==============================] - 1s 831us/step - loss: 1.0858 - accuracy: 0.6375
Epoch 8/60
1298/1298 [==============================] - 1s 975us/step - loss: 1.0813 - accuracy: 0.6399
Epoch 9/60
1298/1298 [==============================] - 1s 846us/step - loss: 1.0673 - accuracy: 0.6465
Epoch 10/60
1298/1298 [==============================] - 1s 863u

1298/1298 [==============================] - 1s 915us/step - loss: 0.9207 - accuracy: 0.6868
Epoch 37/60
1298/1298 [==============================] - 1s 881us/step - loss: 0.9179 - accuracy: 0.6888
Epoch 38/60
1298/1298 [==============================] - 1s 902us/step - loss: 0.9119 - accuracy: 0.6897
Epoch 39/60
1298/1298 [==============================] - 1s 893us/step - loss: 0.9081 - accuracy: 0.6897
Epoch 40/60
1298/1298 [==============================] - 1s 876us/step - loss: 0.9061 - accuracy: 0.6892
Epoch 41/60
1298/1298 [==============================] - 1s 904us/step - loss: 0.9094 - accuracy: 0.6893
Epoch 42/60
1298/1298 [==============================] - 1s 938us/step - loss: 0.8881 - accuracy: 0.6945
Epoch 43/60
1298/1298 [==============================] - 1s 945us/step - loss: 0.8903 - accuracy: 0.6937
Epoch 44/60
1298/1298 [==============================] - 2s 1ms/step - loss: 0.8878 - accuracy: 0.6947
Epoch 45/60
1298/1298 [==============================] - 1s 1ms/step 

In [118]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

639/639 - 1s - loss: 1.1596 - accuracy: 0.6461
Normal Neural Network - Loss: 1.1595799922943115, Accuracy: 0.6461267471313477


In [119]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
prediction_labels

array(['BURGLARY-RESIDENCE', 'LARCENY-FROM VEHICLE', 'ROBBERY-PEDESTRIAN',
       'LARCENY-FROM VEHICLE', 'LARCENY-FROM VEHICLE'], dtype=object)

In [120]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['BURGLARY-RESIDENCE' 'LARCENY-FROM VEHICLE' 'ROBBERY-PEDESTRIAN'
 'LARCENY-FROM VEHICLE' 'LARCENY-FROM VEHICLE']
Actual Labels: ['AUTO THEFT', 'LARCENY-FROM VEHICLE', 'ROBBERY-PEDESTRIAN', 'LARCENY-FROM VEHICLE', 'AUTO THEFT']


# Save the Model

In [121]:
import keras

In [122]:
model.save('../models/DL/deep_learning_200_200_200_10')

INFO:tensorflow:Assets written to: ../models/DL/deep_learning_200_200_200_10/assets


In [164]:
#X_scaler = MinMaxScaler()
#X_scaled_file = np.load('../models/DL/dl_xscaler.npy',allow_pickle=True)
my_scaler = joblib.load('../models/DL/scaler.sav')

In [166]:
#my_scaler.inverse_transform(df.head(1))

In [167]:
df.head(1)

,Beat,Occur Time,Neighborhood,Location Type,Latitude,Longitude,Occur Year,Occur Month,Occur Day,Occur DayofWeek,UCR Literal
0,206.0,57,Buckhead,99.0,33.83752,-84.37574,2009.0,1.0,1.0,3.0,ROBBERY-PEDESTRIAN


In [147]:
X_scaled

array([[ 1.18359375e+00,  2.37500000e-01,  1.00000000e+00,
         1.01020408e+00,  2.62873322e+02, -1.57056181e+03,
         2.01500000e+02,  9.09090909e-02,  3.33333333e-02,
         5.00000000e-01]])

In [154]:
pd.DataFrame([606.0, 570.0, 4.0, 99.0, 35.83752, -85.37574, 2015.0, 1.0, 1.0, 3.0])

,0
0,606.00000
1,570.00000
2,4.00000
3,99.00000
4,35.83752
5,-85.37574
6,2015.00000
7,1.00000
8,1.00000
9,3.00000


In [170]:
#X_unscaled = np.array([604.0, 1630.0, 0.0, 21.0, 33.76142, -84.36492, 2013.0, 12.0, 16.0, 0.0])
X_unscaled = np.array([606.0, 570.0, 4.0, 99.0, 35.83752, -85.37574, 2015.0, 1.0, 1.0, 3.0])

X_scaled = my_scaler.transform([606.0, 570.0, 4.0, 99.0, 35.83752, -85.37574, 2015.0, 1.0, 1.0, 3.0])#X_unscaled * X_scaled_file

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
X_scaled = X_scaled.reshape(1,10)# Unfortunately needs to be reshaped from (,10) to (1,10)
reconstructed_model = keras.models.load_model("../models/DL/deep_learning_100_100_10", compile=True)
encoded_predictions = reconstructed_model.predict_classes(X_scaled)
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('../models/DL/dl_classes.npy',allow_pickle=True)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
prediction_labels

In [106]:
encoded_predictions

array([6])